## SNOTEL

In [1]:
import requests

In [4]:
baseurl = "https://wcc.sc.egov.usda.gov/awdbRestApi/"

parameters

* `beginDate` - yyyy-MM-dd HH:mm
* `centralTendencyType` - MEDIAN
* `duration` - DAILY
* `elements` - PREC, WTEQ, SMS:*, PREC::2
* `endDate` - yyyy-MM-dd HH:mm
* `stationTriplets` - *:*:*

In [10]:
payload = {
    'beginDate': '2024-02-01',
    'endDate': '2024-02-02',
    'centralTendencyType': 'MEDIAN',
    'duration': 'DAILY',
    'elements': 'PREC, WTEQ, SMS:*, PREC::2',
    'stationTriplets': '*:CA:SNTL'
}

In [11]:
r = requests.get(baseurl + 'services/v1/data', params=payload)

In [12]:
r.text

'[{"stationTriplet":"301:CA:SNTL","data":[{"stationElement":{"elementCode":"PREC","ordinal":1,"durationName":"DAILY","dataPrecision":1,"storedUnitCode":"in","originalUnitCode":"in","beginDate":"1984-10-01 00:00:00.0","endDate":"2100-01-01 00:00:00.0"},"values":[{"date":"2024-02-01","value":10.4,"median":12.4},{"date":"2024-02-02","value":10.6,"median":12.6}]},{"stationElement":{"elementCode":"WTEQ","ordinal":1,"durationName":"DAILY","dataPrecision":1,"storedUnitCode":"in","originalUnitCode":"in","beginDate":"1984-10-01 00:00:00.0","endDate":"2100-01-01 00:00:00.0"},"values":[{"date":"2024-02-01","value":7.8,"median":8.8},{"date":"2024-02-02","value":8.0,"median":8.6}]}]},{"stationTriplet":"356:CA:SNTL","data":[{"stationElement":{"elementCode":"PREC","ordinal":1,"durationName":"DAILY","dataPrecision":1,"storedUnitCode":"in","originalUnitCode":"in","beginDate":"1980-07-16 00:00:00.0","endDate":"2100-01-01 00:00:00.0"},"values":[{"date":"2024-02-01","value":13.2,"median":20.4},{"date":"20